# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = "../output_data/clean_city_data.csv"

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Alfred,-33.5906,26.8910,54.18,77,76,1.01,ZA,1659199402
1,Bredasdorp,-34.5322,20.0403,50.11,88,93,7.43,ZA,1659199404
2,Chui,-33.6971,-53.4616,59.09,57,0,2.37,UY,1659199407
3,Fes,34.0372,-4.9998,104.25,19,40,8.05,MA,1659199409
4,Ushuaia,-54.8000,-68.3000,44.22,75,40,21.85,AR,1659199214
...,...,...,...,...,...,...,...,...,...
570,Lixouri,38.2019,20.4314,87.80,58,0,13.80,GR,1659200720
571,Arezzo,43.4724,11.8690,85.15,42,37,7.58,IT,1659200719
572,Kingisepp,59.3733,28.6134,62.67,70,21,4.70,RU,1659200724
573,Kibi,6.1649,-0.5538,78.73,68,98,3.69,GH,1659200727


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [4]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"]

In [5]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
111,Bayir,37.2687,28.2167,77.52,28,0,3.51,TR,1659199658
123,Lagoa,39.0500,-27.9833,73.45,69,0,4.56,PT,1659199686
126,Laguna,38.4210,-121.4238,77.31,68,0,8.05,US,1659199513
146,Pitsunda,43.1592,40.3378,77.18,87,0,5.26,GE,1659199739
193,Chuzhou,32.3219,118.2978,76.78,96,0,4.47,CN,1659199847
314,Ryōtsu-minato,38.0833,138.4333,78.49,82,0,0.83,JP,1659200128
412,Bereda,43.2686,-7.5406,72.14,79,0,9.01,ES,1659200348
452,Saint-Pierre,-21.3393,55.4781,70.92,75,0,8.05,RE,1659200448
557,Weinan,34.5036,109.5089,77.05,70,0,3.22,CN,1659200690


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

Issues with finding a hotel at Bayir


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
111,Bayir,37.2687,28.2167,77.52,28,0,3.51,TR,1659199658,NA
123,Lagoa,39.0500,-27.9833,73.45,69,0,4.56,PT,1659199686,Graciosa Resort - Biosphere Island Hotel
126,Laguna,38.4210,-121.4238,77.31,68,0,8.05,US,1659199513,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
146,Pitsunda,43.1592,40.3378,77.18,87,0,5.26,GE,1659199739,ОТЕЛЬ АБНА
193,Chuzhou,32.3219,118.2978,76.78,96,0,4.47,CN,1659199847,Wyndham Grand Plaza Royale Kaidi
314,Ryōtsu-minato,38.0833,138.4333,78.49,82,0,0.83,JP,1659200128,Sadokamoko Onsen Kohanno Yado Yoshidaya
412,Bereda,43.2686,-7.5406,72.14,79,0,9.01,ES,1659200348,Casa Grande de Anllo
452,Saint-Pierre,-21.3393,55.4781,70.92,75,0,8.05,RE,1659200448,"Villa Delisle, Hotel & Spa"
557,Weinan,34.5036,109.5089,77.05,70,0,3.22,CN,1659200690,Jianguo Hotel


In [8]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
123,Lagoa,39.0500,-27.9833,73.45,69,0,4.56,PT,1659199686,Graciosa Resort - Biosphere Island Hotel
126,Laguna,38.4210,-121.4238,77.31,68,0,8.05,US,1659199513,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
146,Pitsunda,43.1592,40.3378,77.18,87,0,5.26,GE,1659199739,ОТЕЛЬ АБНА
193,Chuzhou,32.3219,118.2978,76.78,96,0,4.47,CN,1659199847,Wyndham Grand Plaza Royale Kaidi
314,Ryōtsu-minato,38.0833,138.4333,78.49,82,0,0.83,JP,1659200128,Sadokamoko Onsen Kohanno Yado Yoshidaya
412,Bereda,43.2686,-7.5406,72.14,79,0,9.01,ES,1659200348,Casa Grande de Anllo
452,Saint-Pierre,-21.3393,55.4781,70.92,75,0,8.05,RE,1659200448,"Villa Delisle, Hotel & Spa"
557,Weinan,34.5036,109.5089,77.05,70,0,3.22,CN,1659200690,Jianguo Hotel


In [9]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

8

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map


# Display figure


In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))